In [7]:
import os
print(f"Current working folder: {os.getcwd()}")

os.chdir('D:/My Document/Khóa Luận Tốt Nghiệp/Model Reposity/RAG_LLM_DA')
print(f"Current working folder: {os.getcwd()}")

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from openai_llm.llm_init import LLM_Model
from utils import load_json_data, save_json_data

Current working folder: D:\My Document\Khóa Luận Tốt Nghiệp\Model Reposity\RAG_LLM_DA
Current working folder: D:\My Document\Khóa Luận Tốt Nghiệp\Model Reposity\RAG_LLM_DA


In [8]:
llm_instance = LLM_Model()

In [9]:
relations = list(load_json_data('datasets/icews14/relation2id.json').keys())


def transform_relations(all_rels, llm_instance, output_dir='./'):
    batch_size = 10

    result = dict()

    system_msg_content = '''
    You are an expert in Temporal Knowledge Graph. Your task is to transform the given relations into a natural language term.
    - Each given relation represents for an action between two entities in the past.
    - Remove underscores and technical formatting  
    - An important note is that if the relation has a prefix 'inv', it means that the action is an inverse action. You need to consider this information when transforming the relation.

    Your answer should be in a json format as below:
    {{
        'original_relations': // list of original relations
        'transformed_relations': // list of transformed relations corresponding to the original relations
    }}
    '''
    system_msg = SystemMessage(content=system_msg_content)

    for i in range(0, len(all_rels), batch_size):
        user_msg_content = f'''
            The list of relations need to be transformed: {all_rels[i:i+batch_size]}
        '''
        user_msg = HumanMessage(content=user_msg_content)

        answer_llm = llm_instance.run_task([system_msg, user_msg])

        for k, v in zip(answer_llm['original_relations'], answer_llm['transformed_relations']):
            result[k] = v
    
    save_json_data(result, os.path.join(output_dir, 'transformed_relations.json'))

Loading data from datasets/icews14/relation2id.json


In [10]:
transform_relations(relations, llm_instance)

Data has been converted to JSON and saved to ./transformed_relations.json
